In [1]:
from toolbox import ToolBox
from tqdm import tqdm

import langid
from models import *

tqdm.pandas()

In [2]:
tb = ToolBox()

In [3]:
df = tb.load_data_sql(table='user_reviews')
df = df[~df['review'].isnull()]
df.head()

,game,source,date,grade,review,lang
0,007-legends-pc,GamingXP,2012-12-03,35.0,"Unfortunately the game proves the fact, that d...",None
1,007-legends-pc,LEVEL (Czech Republic),2012-12-21,30.0,The worst gift agent 007 could possibly get to...,None
2,007-legends-pc,PC Gamer,2013-01-05,16.0,"Legends, you're the worst. [Feb 2013, p.79]",None
3,007-legends-pc,PC Master (Greece),2013-02-06,28.0,"Why put yourself through this ordeal, when the...",None
4,007-legends-pc,Riot Pixels,2013-01-29,25.0,007 Legends greets you with a warning that Act...,None


In [4]:
df.shape

(322070, 6)

In [5]:
def detect_language(review):
    return langid.classify(review)[0]

df['lang'] = df['review'].progress_apply(detect_language)

df.head()

100%|██████████| 322070/322070 [04:20<00:00, 1234.04it/s]


,game,source,date,grade,review,lang
0,007-legends-pc,GamingXP,2012-12-03,35.0,"Unfortunately the game proves the fact, that d...",en
1,007-legends-pc,LEVEL (Czech Republic),2012-12-21,30.0,The worst gift agent 007 could possibly get to...,en
2,007-legends-pc,PC Gamer,2013-01-05,16.0,"Legends, you're the worst. [Feb 2013, p.79]",en
3,007-legends-pc,PC Master (Greece),2013-02-06,28.0,"Why put yourself through this ordeal, when the...",en
4,007-legends-pc,Riot Pixels,2013-01-29,25.0,007 Legends greets you with a warning that Act...,en


In [59]:
def update_lang(review):
    (UserReviews
        .update(lang = review['lang'])
        .where(
            (UserReviews.game == review['game']) & 
            (UserReviews.username == review['username'])).execute())

df.progress_apply(update_lang, axis=1);

100%|██████████| 228445/228445 [1:58:38<00:00, 32.09it/s]   
